Three files have disease info:  

disease-gene.csv

disease-therapeuticarea.csv

drug-disease.csv

Uses ORPHA, HPO, and MONDO (or other) disease

ORPHA we can keep

HPO is not a disease, so make it a separate problem



# map id to disease

for DOID, the path is:
DOID -> BIoPortal DOID Record (get label), follow mappings -> look for the SNOMEDCT mapping -> get ID

In [82]:
require "rest-client"
require 'json'


def map_id_to_disease(name)
  return false unless (name =~ /DOID_(.*)/ || name =~ /MONDO_(.*)/ || name =~ /Orphanet_(.*)/)
  identifier = $1
  api_key = ENV["APIKEY"] # Replace with BioPortal API key
  begin
    if name =~ /DOID/
      url =  "https://data.bioontology.org/search?q=DOID:#{identifier}&ontologies=DOID&require_exact_match=true&apikey=#{api_key}"
    elsif name =~ /MONDO/
      url =  "https://data.bioontology.org/search?q=MONDO:#{identifier}&ontologies=MONDO&require_exact_match=true&apikey=#{api_key}"      
    else
      url =  "https://data.bioontology.org/search?q=Orphanet:#{identifier}&ontologies=ORDO&require_exact_match=true&apikey=74027bd8-6be0-4329-be22-aa3717f97243"
    end
#     warn url
    response = RestClient.get(url)
  rescue StandardError => e
    warn "No data found for #{name} #{e.inspect}\n"
    return false
  end
  mappings = []
  data = JSON.parse(response)

#  if hit = data.dig('collection', 0)
  if hit = data.dig('collection')
    hit.each do |h|
      disease_name = h&.dig('prefLabel')
      uri = h&.dig('@id')
      linksurl = h&.dig('links', 'mappings')
      mappings << { id: name, uri: uri, disease_name: disease_name, linksurl: linksurl}
    end
    return mappings
  else
    warn "No data found for #{name}\n"
    return false
  end

rescue StandardError => e
  warn "No data found for #{name} Error: #{e.inspect}\n"
  return false
end

:map_id_to_disease

In [83]:
# test
["DOID_7551", "MONDO_0003243", "Orphanet_101010"].each do |name|
  warn map_id_to_disease(name)
end
  

{:id=>"DOID_7551", :uri=>"http://purl.obolibrary.org/obo/DOID_7551", :disease_name=>"gonorrhea", :linksurl=>"https://data.bioontology.org/ontologies/DOID/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FDOID_7551/mappings"}
{:id=>"MONDO_0003243", :uri=>"http://purl.obolibrary.org/obo/MONDO_0003243", :disease_name=>"hepatocellular clear cell carcinoma", :linksurl=>"https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0003243/mappings"}
{:id=>"Orphanet_101010", :uri=>"http://www.orpha.net/ORDO/Orphanet_101010", :disease_name=>"Autosomal spastic paraplegia type 30", :linksurl=>"https://data.bioontology.org/ontologies/ORDO/classes/http%3A%2F%2Fwww.orpha.net%2FORDO%2FOrphanet_101010/mappings"}


["DOID_7551", "MONDO_0003243", "Orphanet_101010"]

In [95]:
def get_snomed_from_mapping(mapping)
  if mapping =~ /MONDO/ 
    type = "MONDO"
  else
    type = "DOID_ORDO"  # mapping =~ /Orphanet/
  end

  api_key = ENV["APIKEY"] # Replace with BioPortal API key
  begin
    url = mapping+"?apikey=#{api_key}"
#     warn url
    response = RestClient.get(url)
  rescue StandardError => e
    warn "No data found for mapping #{e.inspect}\n"
    return false
  end
  mappings = []
  data = JSON.parse(response)

  if type == "DOID_ORDO"
    data.each do |mapp|
      mapp["classes"].each do |thisclass|
        next unless thisclass["@id"] =~ /SNOMEDCT/
        return {"snomed": thisclass["@id"]}
      end
    end
    warn "no SNOMED found for DOID/ORDO #{mapping}"
    return false
  else
    data.each do |mapp|
      mapp["classes"].each do |thisclass|
        next unless thisclass["@id"] =~ /(DOID_\d+)/  # if we start with MONDO, thre is no snomed,but there is a DOID
        doid = $1
        results = map_id_to_disease(doid) # go back to the beginning
        if results
          results = results.first # (can only be one doid)
          snomedres = get_snomed_from_mapping(results[:linksurl])  # recursive, but this time its a DOID
          if snomedres
            return snomedres  # {"snomed": thisclass["@id"]} from the routine above
          else
            warn "found no snomed for #{mapping}"
            return false
          end
        else
          warn "found no results for xreffed doid #{doid}"
          return false
        end
      end
    end
  end
    
  warn "No SNOMED found in #{mapping}\n"
  return false
rescue StandardError => e
  warn  "parsing or other error in #{mapping} Error: #{e.inspect}\n"
  return false
end

:get_snomed_from_mapping

In [96]:
# test
["DOID_7551", "MONDO_0003243", "Orphanet_101010"].each do |name|
  results = map_id_to_disease(name)
  if results
    results = results.first # (can only be one)
    nextres = get_snomed_from_mapping(results[:linksurl])
    if nextres
      results.merge! nextres
    end
    puts results
  else
    warn "found no results for #{name}"
  end
end


{:id=>"DOID_7551", :uri=>"http://purl.obolibrary.org/obo/DOID_7551", :disease_name=>"gonorrhea", :linksurl=>"https://data.bioontology.org/ontologies/DOID/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FDOID_7551/mappings", :snomed=>"http://purl.bioontology.org/ontology/SNOMEDCT/15628003"}


no SNOMED found for DOID/ORDO https://data.bioontology.org/ontologies/DOID/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FDOID_5016/mappings
found no snomed for https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0003243/mappings


{:id=>"MONDO_0003243", :uri=>"http://purl.obolibrary.org/obo/MONDO_0003243", :disease_name=>"hepatocellular clear cell carcinoma", :linksurl=>"https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0003243/mappings"}
{:id=>"Orphanet_101010", :uri=>"http://www.orpha.net/ORDO/Orphanet_101010", :disease_name=>"Autosomal spastic paraplegia type 30", :linksurl=>"https://data.bioontology.org/ontologies/ORDO/classes/http%3A%2F%2Fwww.orpha.net%2FORDO%2FOrphanet_101010/mappings", :snomed=>"http://purl.bioontology.org/ontology/SNOMEDCT/763377006"}


["DOID_7551", "MONDO_0003243", "Orphanet_101010"]

In [100]:
require 'csv'

filelist = ["disease-gene.csv","disease-therapeuticarea.csv"]  # in these two it is called "source"

diseaselist = {}
filelist.each do |csvfile|
  CSV.foreach("./rawdata/#{csvfile}", headers: true) do |row|
    next if row.size < 3

    source = row['source']
    abort unless source
    diseaselist[source] = 1
  end
end

filelist = ["drug-disease.csv"]  # in these two it is called "target"

filelist.each do |csvfile|
  CSV.foreach("./rawdata/#{csvfile}", headers: true) do |row|
    next if row.size < 3

    source = row['target']
    abort unless source
    diseaselist[source] = 1
  end
end

puts diseaselist.length  

9339


In [101]:
# write the mapping file
f = File.open("./mappings/diseases.map", "w")
e = File.open("./mappings/diseases-errors.txt", "w")

f.write CSV.generate_line(["source","snomedct","orpha","prefname"])

diseaselist.each_key do |name|
  warn name
  next if name =~ /HP_/  # not HPO terms
  next if name =~ /EFO_/  # not EFO terms
  next if name =~ /OTAR_/  # not EFO terms
  
  results = map_id_to_disease(name)
  if results
    results = results.first # (can only be one)
    nextres = get_snomed_from_mapping(results[:linksurl])
    if nextres
      results.merge! nextres
    else
      warn "found no snomed for links in #{name}"
      e.write "found no snomed for links in #{name}\n"
    end
    snomed = results[:snomed]
    snomed = "https://fake.snomed/not-found" unless snomed
    if results[:uri] =~ /orpha/
      orpha = results[:uri]
    else
      orpha = "https://fake.orphanet/not-found"
    end
    f.write CSV.generate_line([results[:id],snomed,orpha,results[:disease_name]])
    # {:id=>"Orphanet_101010", :uri=>"http://www.orpha.net/ORDO/Orphanet_101010", :disease_name=>"Autosomal spastic paraplegia type 30", :linksurl=>"https://data.bioontology.org/ontologies/ORDO/classes/http%3A%2F%2Fwww.orpha.net%2FORDO%2FOrphanet_101010/mappings", :snomed=>"http://purl.bioontology.org/ontology/SNOMEDCT/763377006"}
  else
    warn "found no results for #{name}"
    e.write "found no results for #{name}\n"
  end
end

DOID_7551
HP_0000857
HP_0012076
MONDO_0030006
no SNOMED found for DOID/ORDO https://data.bioontology.org/ontologies/DOID/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FDOID_0112117/mappings
found no snomed for https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0030006/mappings
found no snomed for links in MONDO_0030006
HP_0000520
HP_0000802
MONDO_0021116
No SNOMED found in https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0021116/mappings
found no snomed for links in MONDO_0021116
HP_0001591
HP_0002035
HP_0008909
MONDO_0002203
No SNOMED found in https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0002203/mappings
found no snomed for links in MONDO_0002203
MONDO_0004392
No SNOMED found in https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0004392/mappings
found no snomed for links in MOND

found no snomed for links in MONDO_0009374
MONDO_0009816
no SNOMED found for DOID/ORDO https://data.bioontology.org/ontologies/DOID/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FDOID_0110943/mappings
found no snomed for https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0009816/mappings
found no snomed for links in MONDO_0009816
MONDO_0010635
no SNOMED found for DOID/ORDO https://data.bioontology.org/ontologies/DOID/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FDOID_0090094/mappings
found no snomed for https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0010635/mappings
found no snomed for links in MONDO_0010635
MONDO_0018715
No SNOMED found in https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0018715/mappings
found no snomed for links in MONDO_0018715
MONDO_0020854
No SNOMED found in https://data.bioontology.org/ontologies/MONDO/cl

found no snomed for links in MONDO_0002863
MONDO_0009866
No SNOMED found in https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0009866/mappings
found no snomed for links in MONDO_0009866
MONDO_0014369
No SNOMED found in https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0014369/mappings
found no snomed for links in MONDO_0014369
MONDO_0020638
No SNOMED found in https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0020638/mappings
found no snomed for links in MONDO_0020638
MONDO_0054565
No SNOMED found in https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0054565/mappings
found no snomed for links in MONDO_0054565
MONDO_0060758
No SNOMED found in https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0060758/mappings
found no snomed for links in M

found no snomed for links in MONDO_0003158
MONDO_0003865
no SNOMED found for DOID/ORDO https://data.bioontology.org/ontologies/DOID/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FDOID_6367/mappings
found no snomed for https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0003865/mappings
found no snomed for links in MONDO_0003865
MONDO_0013549
No SNOMED found in https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0013549/mappings
found no snomed for links in MONDO_0013549
MONDO_0014464
No SNOMED found in https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0014464/mappings
found no snomed for links in MONDO_0014464
MONDO_0024677
No SNOMED found in https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0024677/mappings
found no snomed for links in MONDO_0024677
MONDO_0029130
No SNOMED found in http

found no snomed for https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0016700/mappings
found no snomed for links in MONDO_0016700
EFO_0000255
HP_0011974
MONDO_0008409
No SNOMED found in https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0008409/mappings
found no snomed for links in MONDO_0008409
MONDO_0036484
no SNOMED found for DOID/ORDO https://data.bioontology.org/ontologies/DOID/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FDOID_0080294/mappings
found no snomed for https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0036484/mappings
found no snomed for links in MONDO_0036484
EFO_0000736
MONDO_0011853
No SNOMED found in https://data.bioontology.org/ontologies/MONDO/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMONDO_0011853/mappings
found no snomed for links in MONDO_0011853
MONDO_0020852
no SNOMED found for DOID/ORDO https://dat

...

Orphanet_295038
No data found for Orphanet_295038 #<RestClient::BadGateway: 502 Bad Gateway>
found no results for Orphanet_295038
Orphanet_295081
No data found for Orphanet_295081 #<RestClient::BadGateway: 502 Bad Gateway>
found no results for Orphanet_295081
Orphanet_314718
Orphanet_64542
No data found for Orphanet_64542 #<RestClient::BadGateway: 502 Bad Gateway>
found no results for Orphanet_64542
Orphanet_79132
no SNOMED found for DOID/ORDO https://data.bioontology.org/ontologies/ORDO/classes/http%3A%2F%2Fwww.orpha.net%2FORDO%2FOrphanet_79132/mappings
found no snomed for links in Orphanet_79132
Orphanet_168552
No data found for mapping #<RestClient::BadGateway: 502 Bad Gateway>
found no snomed for links in Orphanet_168552
Orphanet_217385
No data found for Orphanet_217385 #<RestClient::BadGateway: 502 Bad Gateway>
found no results for Orphanet_217385
Orphanet_300313
No data found for Orphanet_300313 #<RestClient::BadGateway: 502 Bad Gateway>
found no results for Orphanet_300313
Orpha

found no results for Orphanet_91494
Orphanet_99832
No data found for mapping #<RestClient::BadGateway: 502 Bad Gateway>
found no snomed for links in Orphanet_99832
Orphanet_93394
No data found for Orphanet_93394 #<RestClient::BadGateway: 502 Bad Gateway>
found no results for Orphanet_93394
Orphanet_99764
No data found for mapping #<RestClient::BadGateway: 502 Bad Gateway>
found no snomed for links in Orphanet_99764
Orphanet_93365
No data found for Orphanet_93365 #<RestClient::BadGateway: 502 Bad Gateway>
found no results for Orphanet_93365
Orphanet_97362
no SNOMED found for DOID/ORDO https://data.bioontology.org/ontologies/ORDO/classes/http%3A%2F%2Fwww.orpha.net%2FORDO%2FOrphanet_97362/mappings
found no snomed for links in Orphanet_97362
Orphanet_96171
No data found for mapping #<RestClient::BadGateway: 502 Bad Gateway>
found no snomed for links in Orphanet_96171
Orphanet_96063
No data found for Orphanet_96063 #<RestClient::BadGateway: 502 Bad Gateway>
found no results for Orphanet_960

No data found for Orphanet_93217 #<RestClient::BadGateway: 502 Bad Gateway>
found no results for Orphanet_93217
Orphanet_99706
No data found for Orphanet_99706 #<RestClient::BadGateway: 502 Bad Gateway>
found no results for Orphanet_99706
Orphanet_96168
No data found for mapping #<RestClient::BadGateway: 502 Bad Gateway>
found no snomed for links in Orphanet_96168
Orphanet_85324
Orphanet_99004
No data found for mapping #<RestClient::BadGateway: 502 Bad Gateway>
found no snomed for links in Orphanet_99004
Orphanet_99171
No data found for mapping #<RestClient::BadGateway: 502 Bad Gateway>
found no snomed for links in Orphanet_99171
Orphanet_91397
No data found for Orphanet_91397 #<RestClient::BadGateway: 502 Bad Gateway>
found no results for Orphanet_91397
Orphanet_96195
No data found for Orphanet_96195 #<RestClient::BadGateway: 502 Bad Gateway>
found no results for Orphanet_96195
Orphanet_85194
Orphanet_99941
Orphanet_96103
No data found for Orphanet_96103 #<RestClient::BadGateway: 502 

Orphanet_99324
no SNOMED found for DOID/ORDO https://data.bioontology.org/ontologies/ORDO/classes/http%3A%2F%2Fwww.orpha.net%2FORDO%2FOrphanet_99324/mappings
found no snomed for links in Orphanet_99324
Orphanet_85284
Orphanet_93400
no SNOMED found for DOID/ORDO https://data.bioontology.org/ontologies/ORDO/classes/http%3A%2F%2Fwww.orpha.net%2FORDO%2FOrphanet_93400/mappings
found no snomed for links in Orphanet_93400
Orphanet_96175
Orphanet_93216
no SNOMED found for DOID/ORDO https://data.bioontology.org/ontologies/ORDO/classes/http%3A%2F%2Fwww.orpha.net%2FORDO%2FOrphanet_93216/mappings
found no snomed for links in Orphanet_93216
Orphanet_93964
no SNOMED found for DOID/ORDO https://data.bioontology.org/ontologies/ORDO/classes/http%3A%2F%2Fwww.orpha.net%2FORDO%2FOrphanet_93964/mappings
found no snomed for links in Orphanet_93964
Orphanet_96074
no SNOMED found for DOID/ORDO https://data.bioontology.org/ontologies/ORDO/classes/http%3A%2F%2Fwww.orpha.net%2FORDO%2FOrphanet_96074/mappings
foun

{"DOID_7551"=>1, "HP_0000857"=>1, "HP_0012076"=>1, "MONDO_0030006"=>1, "HP_0000520"=>1, "HP_0000802"=>1, "MONDO_0021116"=>1, "HP_0001591"=>1, "HP_0002035"=>1, "HP_0008909"=>1, "MONDO_0002203"=>1, "MONDO_0004392"=>1, "MONDO_0012806"=>1, "MONDO_0032911"=>1, "EFO_0000713"=>1, "HP_0009816"=>1, "HP_0010865"=>1, "HP_0012042"=>1, "MONDO_0008597"=>1, "MONDO_0044784"=>1, "MONDO_0054781"=>1, "HP_0002571"=>1, "HP_0004308"=>1, "HP_0012390"=>1, "HP_0012410"=>1, "HP_0100727"=>1, "MONDO_0009619"=>1, "MONDO_0011377"=>1, "MONDO_0015003"=>1, "MONDO_0032889"=>1, "MONDO_0000030"=>1, "MONDO_0030060"=>1, "EFO_0001675"=>1, "HP_0000327"=>1, "HP_0006789"=>1, "HP_0030834"=>1, "MONDO_0001410"=>1, "MONDO_0014725"=>1, "MONDO_0054695"=>1, "EFO_0000174"=>1, "EFO_0003073"=>1, "HP_0000200"=>1, "MONDO_0012670"=>1, "MONDO_0014608"=>1, "MONDO_0019471"=>1, "MONDO_0044325"=>1, "EFO_0003075"=>1, "HP_0002904"=>1, "EFO_0000595"=>1, "HP_0002153"=>1, "MONDO_0009559"=>1, "MONDO_0015643"=>1, "MONDO_0017386"=>1, "MONDO_0030019"=>1